## BPC/SU/II Survey

### Install Packages, Load Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from textwrap import wrap

#set location for saving files
directory = "C://Users//rorey//OneDrive - Bipartisan Policy Center//Elections Project//Research//BPC SU II Joint Survey Dec 23//programming//files//"

In [2]:
codebook = pd.read_csv('https://raw.githubusercontent.com/rachelorey/Dec23-survey/main/2312082_BPC_levels_codebook.csv', encoding='utf-8')
data = pd.read_csv('https://raw.githubusercontent.com/rachelorey/Dec23-survey/main/2312082_BPC_raw_data.csv', encoding='utf-8')
q_codebook = pd.read_csv('https://raw.githubusercontent.com/rachelorey/Dec23-survey/main/2312082_BPC_question_codebook.csv', encoding='utf-8', index_col = 'qid')

# q_codebook into dict
q_codebook = q_codebook.to_dict().get('qidFull')

# display(data)

In [3]:
nulldata = pd.DataFrame(data.isnull().sum(),columns=["Sum_Null"]) ##2203 rows in entire dataset
nulldata.sort_values(by=["Sum_Null"],ascending=False,inplace=True)

# nulldata[nulldata["Sum_Null"]>0].head(20)

## Get Percents by Demo for Scaled Questions
#### BPC21, BPC3

In [4]:
def weighted_counts(x):
    # x is a DataFrame of grouped values including the 'wts' column for weights
    total_weight = x['wts'].sum()
    return total_weight

def getdemo_scaled(demo,q,collapse_scales=False):
    # Extracting demo labels
    labels = dict(zip(codebook[codebook['question'] == demo]['value'], codebook[codebook['question'] == demo]['code']))

    # Filtering out the relevant rows from the codebook for BPCX responses
    bpcx_codebook = codebook[codebook['question'].str.contains(q)]

    # Creating a dictionary to map response values to their meanings
    value_to_meaning = dict(zip(bpcx_codebook['value'], bpcx_codebook['code']))

    # List to hold the reshaped tables
    long_format_tables = []

    # Iterate through each BPCX question
    for question in data.filter(regex='^'+q).columns:

##ADDING WEIGHTS
        # Group by 'demo' and 'question', then apply the weighted counts calculation
        summary_table = data.groupby([demo,question]).apply(weighted_counts).unstack()
    
        # Rename index using the labels dictionary
        summary_table = summary_table.rename(index=labels)

        # Rename columns using the value_to_meaning dictionary
        summary_table = summary_table.rename(columns=value_to_meaning)

        # Reset index to make demo a column
        summary_table = summary_table.reset_index()

        # Melt the DataFrame to long format
        melted = summary_table.melt(id_vars=demo, var_name='Response', value_name='Count')
        melted['Question'] = question

        # Replace the question shorthand with specific concern
        melted['Q_Text'] = q_codebook.get(question, question)  # Fallback to question ID if not found

        # Append to the list
        long_format_tables.append(melted)

    # Concatenate all the long format tables into one
    combined_long_table = pd.concat(long_format_tables)
    try:
        combined_long_table["Q_Text"] = combined_long_table["Q_Text"].str.split(' --- ', expand=True)[1].fillna(combined_long_table["Q_Text"])
    except:
        pass
    

    response_mapping = {
        'Very concerned': 'Concerned',
        'Somewhat concerned': 'Concerned',
        'Not too concerned': 'Not Concerned',
        'Not at all concerned': 'Not Concerned',
        "Don't know/No opinion": "Don't know/No opinion",
        'Very important': 'Important',
        'Somewhat important': 'Important',
        'Not too important': 'Not important',
        'Not at all important': 'Not important',
        "Far too little":"Too little",
        "Far too much":"Too much"        
    }
    
    if collapse_scales: #if function says to collapse scales (ie take out very, somehwat)
        if ~combined_long_table["Response"].str.lower().str.contains("confident").any(): #if Response values don't include "confident"
            # Apply the mapping to collapse response categories
            combined_long_table['Response'] = combined_long_table['Response'].map(response_mapping)

    # Pivot the table to wide format
    wide_format_table = combined_long_table.pivot_table(index=['Q_Text','Response'], 
                                                        columns=[demo], 
                                                        values='Count',
                                                        fill_value=0)

    wide_format_table.reset_index(inplace=True)

    for q_level in wide_format_table["Q_Text"].unique():
        # Mask for the rows that correspond to the question
        question_mask = wide_format_table["Q_Text"] == q_level

        # Select the relevant data for the current question
        temp = wide_format_table.loc[question_mask, wide_format_table.columns[2:]]

        # Calculate the totals for each column (for the current question)
        totals = temp.sum()

        # Use .loc to update the original dataframe
        wide_format_table.loc[question_mask, temp.columns] = temp / totals.values

    
    # GET TOTAL PERCENTS
    
    # Get total responses by Q_Text
    total_counts = combined_long_table.groupby(['Q_Text'])['Count'].sum().reset_index(name='Total Count')

    overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
    overall_percent.rename({"Count":"Overall"},axis=1,inplace=True)
    
    
    wide_format_table = pd.merge(wide_format_table, overall_percent, on=["Response", 'Q_Text'])
    
    wide_format_table.to_csv(directory+q+"_"+demo+".csv",index=False)
    
#     display(wide_format_table)

    return(combined_long_table,wide_format_table)

In [5]:
# for q in ["BPC12","BPC13","BPC14","BPC16"]:
#     getdemo_scaled("age",q,False)

In [6]:
# getdemo_scaled("xpid3","BPC7",False) #BPC7 BPC5

In [7]:
# for q in ["BPC4","BPC7"]:
#     getdemo_scaled("xpid3",q,False)

In [8]:
# ##RUNS FOR ALL CONFIDENCE QUESTIONS

# res = pd.DataFrame(columns=['Q_Text', 'Response', 'PID: Dem (no lean)', 'PID: Ind (no lean)','PID: Rep (no lean)','Overall'])

# dataframes_to_concat = [res]

# for que in ["BPC17","BPC18","BPC19","BPC20"]:
#     combined_long_table,wide_format_table = getdemo_scaled("xpid3",que,False)
#     dataframes_to_concat.append(wide_format_table)

# res = pd.concat(dataframes_to_concat, ignore_index=True)

# res.to_csv(directory+"BPC17-20.csv",index=False)

## Getting Percents by Question and Demo for "Select All That Apply" Questions

In [9]:
##FUNCTION to get breakdown in responses for "select all that apply" question by demo

def get_demo_percents(demo,q):

    # Extracting political party labels
    demo_labels = dict(zip(codebook[codebook['question'] == demo]['value'], codebook[codebook['question'] == demo]['code']))

    # Filtering out the relevant rows from the codebook for BPCX responses
    bpcx_codebook = codebook[codebook['question'].str.contains(q)]

    # Creating a dictionary to map response values to their meanings
    value_to_meaning = dict(zip(bpcx_codebook['value'], bpcx_codebook['code']))

    # List to hold the reshaped tables
    long_format_tables = []

    # Iterate through each BPCX question
    for question in data.filter(regex='^'+q+"_").columns:

        ##ADDING WEIGHTS
        # Group by 'demo' and 'question', then apply the weighted counts calculation
        summary_table = data.groupby([demo,question]).apply(weighted_counts).unstack()

        # Rename index using the demo_labels dictionary
        summary_table = summary_table.rename(index=demo_labels)

        # Rename columns using the value_to_meaning dictionary
        summary_table = summary_table.rename(columns=value_to_meaning)

        # Reset index to make demo a column
        summary_table = summary_table.reset_index()
    #     display(summary_table)

        # Melt the DataFrame to long format
        melted = summary_table.melt(id_vars=demo, var_name='Response', value_name='Count')
        melted['Question'] = question
    #     display(melted)

        # Replace the question shorthand
        melted['Q_Text'] = q_codebook.get(question, question)  # Fallback to question ID if not found

        # Append to the list
        long_format_tables.append(melted)

    # Concatenate all the long format tables into one
    combined_long_table = pd.concat(long_format_tables)

    #remove "Other" open-text responses
    combined_long_table = combined_long_table[~combined_long_table["Question"].str.contains("TEXT")]

    #remove superfluous question text
    try:
        combined_long_table["Q_Text"] = combined_long_table["Q_Text"].str.split(' --- ', expand=True)[1].fillna(combined_long_table["Q_Text"])
    except:
        pass
    
    combined_long_table.drop(["Question"],axis=1,inplace=True)
    combined_long_table["Response"] = combined_long_table["Response"].str.replace('NO TEXT',"Selected")

    ##GET %s

    # Group by demo and 'Q_Text', then calculate the sum for each group
    total_counts = combined_long_table.groupby([demo, 'Q_Text'])['Count'].sum().reset_index(name='Total Count')

    # Filter only 'Selected' responses and calculate the sum
    selected_counts = combined_long_table[combined_long_table['Response'] == 'Selected'].groupby([demo, 'Q_Text'])['Count'].sum().reset_index(name='Selected Count')

    # Merge the total and selected counts on demo and 'Q_Text'
    merged_counts = pd.merge(total_counts, selected_counts, on=[demo, 'Q_Text'])

    # Calculate the percentage of 'Selected' for each demo and 'Q_Text'
    merged_counts['Selected Percentage'] = (merged_counts['Selected Count'] / merged_counts['Total Count'])
    counts = merged_counts[["Q_Text",demo,"Selected Count"]]
    counts = counts.pivot(index='Q_Text', columns=demo, values='Selected Count').reset_index()
    display(counts)
#     merged_counts.drop(["Total Count","Selected Count"],axis=1,inplace=True)
    merged_counts = merged_counts.pivot(index='Q_Text', columns=demo, values='Selected Percentage').reset_index()
    
    # Get total responses by Q_Text
    total_counts = combined_long_table.groupby(['Q_Text'])['Count'].sum().reset_index(name='Total Count')

    overall_percent = (combined_long_table.groupby(["Response","Q_Text"])['Count'].sum()/combined_long_table.groupby(['Q_Text'])['Count'].sum()[0]).reset_index()
    overall_percent.rename({"Count":"Overall"},axis=1,inplace=True)
    
    #merge
    overall_percent = overall_percent[overall_percent["Response"]=="Selected"]
    overall_percent.drop(["Response"],inplace=True,axis=1)
    
    merged_counts = pd.merge(merged_counts, overall_percent, on=['Q_Text'])
    display(merged_counts)

    merged_counts.to_csv(directory+q+"_"+demo+".csv",index=False)
#     counts.to_csv(directory+q+"_"+demo+"_COUNTS.csv",index=False)
    return(combined_long_table,merged_counts,overall_percent)

In [10]:
# for q in ["BPC8"]:
#     combined_long_table,merged_counts,overall_percent = get_demo_percents("age",q)

In [11]:
# q = "BPC2" #"BPC1"

# for demo in ["xpid3","age","xdemGender"]:
#     get_demo_percents(demo,q)

## Get over all %s (questions with 1 and 2 -- selected/not selected)

In [12]:
def get_weighted_response_totals(data, column, condition):
    """Calculate the sum of weights for responses meeting a specific condition."""
    # Use .loc to ensure the operation is done on the DataFrame, multiplying condition by weights
    return (data[column] == condition) * data['wts']

def get_total_percents(question):
    # Filter columns based on the question prefix
    bpc_columns = data.filter(regex='^'+question+'_').columns
    
    # Initialize dictionaries to store weighted totals and denominators
    weighted_totals = {}
    weighted_denoms = {}
    
    # Calculate weighted totals and denominators for each column
    for col in bpc_columns:
        weighted_totals[col] = get_weighted_response_totals(data, col, 1).sum()
        weighted_denoms[col] = get_weighted_response_totals(data, col, 1).sum() + get_weighted_response_totals(data, col, 2).sum()

    # Convert dictionaries to DataFrame
    BPC = pd.DataFrame(list(weighted_totals.items()), columns=['index', 'total'])
    
    # Calculate denominators for percentages and add them as a new column
    BPC['denominator'] = BPC['index'].map(weighted_denoms)
    
    # Replace question codes with text descriptions
    BPC["index"] = BPC["index"].map(q_codebook)
    try:
        BPC["index"] = BPC["index"].str.split(' --- ', expand=True)[1].fillna(BPC["index"])
    except:
        pass
    
    # Calculate percentages
    BPC["Percent"] = BPC["total"] / BPC['denominator']
    BPC.drop(["total","denominator"],axis=1,inplace=True)
    
    BPC.to_csv(directory+question+"_overall.csv",index=False)
    
    return BPC

In [13]:
# for q in ["BPC1","BPC2","BPC9"]:
#     get_total_percents(q)

In [14]:
# for q in ["BPC11","BPC10"]:
#     get_total_percents(q)

In [15]:
# getdemo_scaled("xpid3","BPC4",False)

## MISC  - get counts (not percent) for BPC12

In [16]:
# combined_long_table,wide_format_table = getdemo_scaled("age","BPC12",False)

# demo='age'

# # Pivot the table to wide format
# wide_format_table = combined_long_table.pivot_table(index=['Q_Text','Response'], 
#                                                     columns=[demo], 
#                                                     values='Count',
#                                                     fill_value=0)

# wide_format_table.reset_index(inplace=True)

# for q_level in wide_format_table["Q_Text"].unique():
#     # Mask for the rows that correspond to the question
#     question_mask = wide_format_table["Q_Text"] == q_level

#     # Select the relevant data for the current question
#     temp = wide_format_table.loc[question_mask, wide_format_table.columns[2:]]

#     # Calculate the totals for each column (for the current question)
#     totals = temp.sum()

#     # Use .loc to update the original dataframe
# #     wide_format_table.loc[question_mask, temp.columns] = temp / totals.values

# wide_format_table.to_csv(directory+"BPC12counts.csv",index=False)

## MISC - BPC3 and BPC21 - get importance levels

In [17]:
# # GET importance levels for BPC21

# combined_long_table,wide_format_table = getdemo_scaled("xpid3","BPC21",False)

# # important = wide_format_table[(wide_format_table["Response"]=="Very important")|(wide_format_table["Response"]=="Somewhat important")]
# important = wide_format_table[(wide_format_table["Response"]=="Very concerned")|(wide_format_table["Response"]=="Somewhat concerned")]

# important = pd.DataFrame(important.groupby("Q_Text").value_counts()).reset_index()

# important = important[['Q_Text', 'Response', 'Overall']]
# important = important.pivot_table(index=['Q_Text'], columns=['Response'], 
#                                                     values='Overall',
#                                                     fill_value=0)

# important.to_csv(directory+"BPC21_important.csv",index=True)

# # GET importance levels for BPC3

# combined_long_table,wide_format_table = getdemo_scaled("xpid3","BPC3",False)

# important = wide_format_table[(wide_format_table["Response"]=="Very important")|(wide_format_table["Response"]=="Somewhat important")]
# # important = wide_format_table[(wide_format_table["Response"]=="Very concerned")|(wide_format_table["Response"]=="Somewhat concerned")]

# important = pd.DataFrame(important.groupby("Q_Text").value_counts()).reset_index()

# important = important[['Q_Text', 'Response', 'Overall']]
# important = important.pivot_table(index=['Q_Text'], columns=['Response'], 
#                                                     values='Overall',
#                                                     fill_value=0)

# important.to_csv(directory+"BPC3_important.csv",index=True)

## MISC - GET TWO DEMOS: AGE AND PID FOR QUESTION 1 & 2

In [18]:
# def weighted_counts(x):
#     # x is a DataFrame of grouped values including the 'wts' column for weights
#     total_weight = x['wts'].sum()
#     return total_weight

# q = "BPC1"
# demo1 = 'age'
# demo2 = 'xpid3'


# # Assume demo1 and demo2 are your two demographic categories

# # Extracting political party labels for both demographic categories
# demo1_labels = dict(zip(codebook[codebook['question'] == demo1]['value'], codebook[codebook['question'] == demo1]['code']))
# demo2_labels = dict(zip(codebook[codebook['question'] == demo2]['value'], codebook[codebook['question'] == demo2]['code']))

# # Combining both sets of labels into a single dictionary for easier access later
# demo_labels = {**demo1_labels, **demo2_labels}

# # Filtering out the relevant rows from the codebook for BPCX responses remains the same
# bpcx_codebook = codebook[codebook['question'].str.contains(q)]

# # Creating a dictionary to map response values to their meanings also remains the same
# value_to_meaning = dict(zip(bpcx_codebook['value'], bpcx_codebook['code']))

# # List to hold the reshaped tables still needed
# long_format_tables = []

# # Iterate through each BPCX question as before
# for question in data.filter(regex='^'+q+'_').columns:
#     # Group by both 'demo1' and 'demo2' along with 'question', then apply the weighted counts calculation
#     summary_table = data.groupby([demo1, demo2, question]).apply(weighted_counts).unstack()

#     # Correctly setting levels for MultiIndex without using 'inplace'
#     summary_table.index = summary_table.index.set_levels([summary_table.index.levels[0].map(demo1_labels.get),
#                                                           summary_table.index.levels[1].map(demo2_labels.get)])
    
#     # Rename columns using the value_to_meaning dictionary remains the same
#     summary_table = summary_table.rename(columns=value_to_meaning)

#     # Reset index to make both demos columns. This is adjusted to accommodate both demographic categories.
#     summary_table = summary_table.reset_index()

#     # Melt the DataFrame to long format, adjusting id_vars to include both demographic categories
#     melted = summary_table.melt(id_vars=[demo1, demo2], var_name='Response', value_name='Count')
#     melted['Question'] = question

#     # Replace the question shorthand remains the same
#     melted['Q_Text'] = q_codebook.get(question, question)  # Fallback to question ID if not found

#     # Append to the list remains the same
#     long_format_tables.append(melted)

# # Concatenate all the long format tables into one remains the same
# combined_long_table = pd.concat(long_format_tables)


In [19]:
# # Remove "Other" open-text responses
# try:
#     combined_long_table = combined_long_table[~combined_long_table["Question"].str.contains("TEXT")]
# except:
#     pass

# # Remove superfluous question text
# try:
#     combined_long_table["Q_Text"] = combined_long_table["Q_Text"].str.split(' --- ', expand=True)[1].fillna(combined_long_table["Q_Text"])
# except:
#     pass

# try:
#     combined_long_table.drop(["Question"], axis=1, inplace=True)
# except:
#     pass

# combined_long_table["Response"] = combined_long_table["Response"].str.replace('NO TEXT', "Selected")

# ##GET %s

# # Group by both demos and 'Q_Text', then calculate the sum for each group
# total_counts = combined_long_table.groupby([demo1, demo2, 'Q_Text'])['Count'].sum().reset_index(name='Total Count')

# # Filter only 'Selected' responses and calculate the sum
# selected_counts = combined_long_table[combined_long_table['Response'] == 'Selected'].groupby([demo1, demo2, 'Q_Text'])['Count'].sum().reset_index(name='Selected Count')

# # Merge the total and selected counts on both demos and 'Q_Text'
# merged_counts = pd.merge(total_counts, selected_counts, on=[demo1, demo2, 'Q_Text'])

# # Calculate the percentage of 'Selected' for each combination of demo1 and demo2 and 'Q_Text'
# merged_counts['Selected Percentage'] = (merged_counts['Selected Count'] / merged_counts['Total Count'] * 100)  # Convert to percentage

# merged_counts.drop(["Total Count", "Selected Count"], axis=1, inplace=True)

# # To handle the two-dimensional structure, you might need to pivot or reshape data differently.
# # If you want to pivot to have a multi-level column structure for demos, you can do something like this:
# merged_counts_pivot = merged_counts.pivot_table(index='Q_Text', columns=[demo1, demo2], values='Selected Percentage').reset_index()

# # Get total responses by Q_Text
# total_counts_by_qtext = combined_long_table.groupby(['Q_Text'])['Count'].sum().reset_index(name='Total Count')

# # Calculate overall percentages
# overall_percent = (combined_long_table[combined_long_table['Response'] == 'Selected'].groupby(['Q_Text'])['Count'].sum() / total_counts_by_qtext['Total Count'][0]).reset_index(name='Selected Percentage')

# # Flatten the MultiIndex columns
# # Convert MultiIndex columns to a simple index by joining level names with an underscore (or any other separator you prefer)
# merged_counts_pivot.columns = ['_'.join(col).strip() for col in merged_counts_pivot.columns.values]
# merged_counts_pivot.rename({"Q_Text_":"Q_Text"},axis=1,inplace=True)

# # Now, both DataFrames should have a single-level column structure
# # Merge the overall percentages with the demo-based percentages
# final_merged_counts = pd.merge(merged_counts_pivot, overall_percent, on=['Q_Text'], how='left')

# # Display the final merged table
# display(final_merged_counts)


# final_merged_counts.to_csv(directory+q+"_age&xpid.csv",index=True)